In [1]:
from azureml.core import Workspace, Experiment

ws = Workspace.from_config()
exp = Experiment(workspace=ws, name="udacity-project")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()


Workspace name: quick-starts-ws-190942
Azure region: southcentralus
Subscription id: 3e42d11f-d64d-4173-af9b-12ecaa1030b3
Resource group: aml-quickstarts-190942


In [2]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

cluster_name = "mlcluster"

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

### YOUR CODE HERE ###

# Verify that cluster does not exist already
try:
    cpu_cluster = ComputeTarget(workspace=ws, name=cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    # To use a different region for the compute, add a location='<region>' parameter
    compute_config = AmlCompute.provisioning_configuration(vm_size='Standard_D2_V2',
                                                           max_nodes=4)
    cpu_cluster = ComputeTarget.create(ws, cluster_name, compute_config)

cpu_cluster.wait_for_completion(show_output=True)



InProgress.
SucceededProvisioning operation finished, operation "Succeeded"
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [3]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import choice, uniform
from azureml.core import Environment, ScriptRunConfig
import os
import shutil

# Specify parameter sampler
#ps = ### YOUR CODE HERE ###
ps = RandomParameterSampling( {
    "--C": uniform(0.5, 50), 
    "--max_iter": choice(75, 100, 125)
    }
)

# Specify a Policy
#policy = ### YOUR CODE HERE ###
policy = BanditPolicy(slack_factor = 0.1, evaluation_interval=1, delay_evaluation=5)

if "training" not in os.listdir():
    os.mkdir("./training")

shutil.move('./train.py', './training')

# # Create a SKLearn estimator for use with train.py
# est = ScriptRunConfig(source_directory='./training',
#                     script='train.py',
#                     compute_target=compute_target,
#                     environment=sklearn_env)  


# Setup environment for your training run
sklearn_env = Environment.from_conda_specification(name='sklearn-env', file_path='conda_dependencies.yml')

# Create a ScriptRunConfig Object to specify the configuration details of your training job
#src = ### YOUR CODE HERE ###
# configure and submit your training run
src = ScriptRunConfig(source_directory='./training',
                            script='train.py',
                            compute_target=cpu_cluster,
                            environment=sklearn_env)

# Create a HyperDriveConfig using the src object, hyperparameter sampler, and policy.
# hyperdrive_config = ### YOUR CODE HERE ###
hyperdrive_config = HyperDriveConfig(run_config=src,
                             hyperparameter_sampling=ps,
                             policy=policy,
                             primary_metric_name="Accuracy",
                             primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                             max_total_runs=30,
                             max_concurrent_runs=4)

In [5]:
# Submit your hyperdrive run to the experiment and show run details with the widget.

### YOUR CODE HERE ###
hyperdrive_run = exp.submit(hyperdrive_config)

RunDetails(hyperdrive_run).show()
hyperdrive_run.wait_for_completion(show_output=True)


_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

RunId: HD_f7fadf62-5820-466e-b68e-94aafca60b2f
Web View: https://ml.azure.com/runs/HD_f7fadf62-5820-466e-b68e-94aafca60b2f?wsid=/subscriptions/3e42d11f-d64d-4173-af9b-12ecaa1030b3/resourcegroups/aml-quickstarts-190942/workspaces/quick-starts-ws-190942&tid=660b3398-b80e-49d2-bc5b-ac1dc93b5254

Streaming azureml-logs/hyperdrive.txt

"<START>[2022-04-04T03:11:19.242114][API][INFO]Experiment created<END>\n""<START>[2022-04-04T03:11:19.973228][GENERATOR][INFO]Trying to sample '4' jobs from the hyperparameter space<END>\n"

Execution Summary
RunId: HD_f7fadf62-5820-466e-b68e-94aafca60b2f
Web View: https://ml.azure.com/runs/HD_f7fadf62-5820-466e-b68e-94aafca60b2f?wsid=/subscriptions/3e42d11f-d64d-4173-af9b-12ecaa1030b3/resourcegroups/aml-quickstarts-190942/workspaces/quick-starts-ws-190942&tid=660b3398-b80e-49d2-bc5b-ac1dc93b5254



{'runId': 'HD_f7fadf62-5820-466e-b68e-94aafca60b2f',
 'target': 'mlcluster',
 'status': 'Completed',
 'startTimeUtc': '2022-04-04T03:11:19.028544Z',
 'endTimeUtc': '2022-04-04T03:28:01.947533Z',
 'services': {},
 'properties': {'primary_metric_config': '{"name": "Accuracy", "goal": "maximize"}',
  'resume_from': 'null',
  'runTemplate': 'HyperDrive',
  'azureml.runsource': 'hyperdrive',
  'platform': 'AML',
  'ContentSnapshotId': '163124a8-849c-4394-b2d0-a69169d377cf',
  'user_agent': 'python/3.8.5 (Linux-5.4.0-1073-azure-x86_64-with-glibc2.10) msrest/0.6.21 Hyperdrive.Service/1.0.0 Hyperdrive.SDK/core.1.39.0',
  'space_size': 'infinite_space_size',
  'score': '0.9093226511289147',
  'best_child_run_id': 'HD_f7fadf62-5820-466e-b68e-94aafca60b2f_22',
  'best_metric_status': 'Succeeded'},
 'inputDatasets': [],
 'outputDatasets': [],
 'logFiles': {'azureml-logs/hyperdrive.txt': 'https://mlstrg190942.blob.core.windows.net/azureml/ExperimentRun/dcid.HD_f7fadf62-5820-466e-b68e-94aafca60b2f/a

KeyError: 'log_files'

In [6]:
import joblib
# Get your best run and save the model from that run.

### YOUR CODE HERE ###
best_run = hyperdrive_run.get_best_run_by_primary_metric()
best_run_metrics = best_run.get_metrics()
parameter_values = best_run.get_details()['runDefinition']['arguments']

print('Best Run Id: ', best_run.id)
print('Accuracy:', best_run_metrics['Accuracy'])
print("Parameter " + parameter_values[0] + " : ",parameter_values[1])
print("Parameter " + parameter_values[2] + " : ",parameter_values[3])

print("Best Run Metrics:",best_run.get_metrics())
print("Best run file names :",best_run.get_file_names())

model = best_run.register_model(model_name='hyperdrive-best-model', model_path='outputs/model.joblib', description='Best Run for the Hyperdrive experiment', tags=None)

Best Run Id:  HD_f7fadf62-5820-466e-b68e-94aafca60b2f_22
Accuracy: 0.9093226511289147
Parameter --C :  34.03478170436071
Parameter --max_iter :  125
Best Run Metrics: {'Regularization Strength:': 34.03478170436071, 'Max iterations:': 125, 'Accuracy': 0.9093226511289147}
Best run file names : ['logs/azureml/dataprep/backgroundProcess.log', 'logs/azureml/dataprep/backgroundProcess_Telemetry.log', 'logs/azureml/dataprep/rslex.log', 'logs/azureml/dataprep/rslex.log.2022-04-04-03', 'outputs/model.joblib', 'system_logs/cs_capability/cs-capability.log', 'system_logs/hosttools_capability/hosttools-capability.log', 'system_logs/lifecycler/execution-wrapper.log', 'system_logs/lifecycler/lifecycler.log', 'system_logs/lifecycler/vm-bootstrapper.log', 'user_logs/std_log.txt']


In [7]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

### YOUR CODE HERE ###
url_path = "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"
ds = TabularDatasetFactory.from_delimited_files(path=url_path)

In [8]:
from train import clean_data
from sklearn.model_selection import train_test_split
import pandas as pd

# Use the clean_data function to clean your data.
# x and y are dataframes
x, y = clean_data(ds)
df = pd.concat([x, y], axis=1)

train_data, test_data = train_test_split(df, test_size=0.20, random_state=50)

In [9]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task='classification',
    primary_metric='accuracy',
    training_data=train_data,  
    label_column_name='y',
    n_cross_validations=2)

In [10]:
# Submit your automl run

### YOUR CODE HERE ###
automl_run = exp.submit(automl_config)

RunDetails(automl_run).show()
automl_run.wait_for_completion(show_output=True)

2022-04-04:03:34:48,92 INFO     [modeling_bert.py:226] Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex .
2022-04-04:03:34:48,118 INFO     [modeling_xlnet.py:339] Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex .
2022-04-04:03:34:55,337 WARNING  [local_experiment_driver.py:146] Running on local machine. Note that local runs always run synchronously even if you use the parameter 'show_output=False'


Experiment,Id,Type,Status,Details Page,Docs Page
udacity-project,AutoML_ce983c4f-174d-4850-919b-b9eb119c9129,automl,Preparing,Link to Azure Machine Learning studio,Link to Documentation


2022-04-04:04:06:02,108 INFO     [explanation_client.py:334] Using default datastore for uploads


_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

Experiment,Id,Type,Status,Details Page,Docs Page
udacity-project,AutoML_ce983c4f-174d-4850-919b-b9eb119c9129,automl,Completed,Link to Azure Machine Learning studio,Link to Documentation




********************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       ALERTED
DESCRIPTION:  To decrease model bias, please cancel the current run and fix balancing problem.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData
DETAILS:      Imbalanced data can lead to a falsely perceived positive effect of a model's accuracy because the input data has bias towards one class.
+------------------------------+--------------------------------+--------------------------------------+
|Size of the smallest class    |Name/Label of the smallest class|Number of samples in the training data|
+==============================+================================+======================================+
|2923                          |1                               |26360                                 |
+------------------------------+--------------------------------+----

{'runId': 'AutoML_ce983c4f-174d-4850-919b-b9eb119c9129',
 'target': 'local',
 'status': 'Completed',
 'startTimeUtc': '2022-04-04T03:34:56.11349Z',
 'endTimeUtc': '2022-04-04T04:03:42.923506Z',
 'services': {},
 'warnings': [{'source': 'JasmineService',
   'message': 'No scores improved over last 20 iterations, so experiment stopped early. This early stopping behavior can be disabled by setting enable_early_stopping = False in AutoMLConfig for notebook/python SDK runs.'}],
 'properties': {'num_iterations': '1000',
  'training_type': 'TrainFull',
  'acquisition_function': 'EI',
  'primary_metric': 'accuracy',
  'train_split': '0',
  'acquisition_parameter': '0',
  'num_cross_validation': '2',
  'target': 'local',
  'AMLSettingsJsonString': '{"path":null,"name":"udacity-project","subscription_id":"3e42d11f-d64d-4173-af9b-12ecaa1030b3","resource_group":"aml-quickstarts-190942","workspace_name":"quick-starts-ws-190942","region":"southcentralus","compute_target":"local","spark_service":null

In [13]:
# Retrieve and save your best automl model.

### YOUR CODE HERE ###
best_run, fitted_model = automl_run.get_output()

#best_run = automl_run.get_best_child()
print(best_run)
print("Best run metrics :",best_run.get_metrics())
print("Best run details :",best_run.get_details())
print(fitted_model.steps)

model = automl_run.register_model(model_name = 'automl-best-model')


Run(Experiment: udacity-project,
Id: AutoML_ce983c4f-174d-4850-919b-b9eb119c9129_35,
Type: None,
Status: Completed)
Best run metrics : {'AUC_macro': 0.9476899599567205, 'recall_score_micro': 0.9175644916540213, 'f1_score_micro': 0.9175644916540213, 'average_precision_score_weighted': 0.9550915425209148, 'precision_score_macro': 0.7963127755679125, 'average_precision_score_micro': 0.9816668217830958, 'f1_score_weighted': 0.9152654106721063, 'precision_score_weighted': 0.91359580096617, 'balanced_accuracy': 0.765043167725721, 'weighted_accuracy': 0.9550329988619899, 'accuracy': 0.9175644916540213, 'recall_score_macro': 0.765043167725721, 'precision_score_micro': 0.9175644916540213, 'average_precision_score_macro': 0.8214180346265896, 'AUC_micro': 0.9809739345032363, 'f1_score_macro': 0.7794424017628957, 'AUC_weighted': 0.9476899599567203, 'norm_macro_recall': 0.530086335451442, 'matthews_correlation': 0.5604821682596908, 'recall_score_weighted': 0.9175644916540213, 'log_loss': 0.26603628

In [14]:
print(fitted_model.steps)

[('datatransformer', DataTransformer(enable_dnn=False, enable_feature_sweeping=True, feature_sweeping_config={}, feature_sweeping_timeout=86400, featurization_config=None, force_text_dnn=False, is_cross_validation=True, is_onnx_compatible=False, task='classification')), ('prefittedsoftvotingclassifier', PreFittedSoftVotingClassifier(classification_labels=array([0, 1]), estimators=[('24', Pipeline(memory=None, steps=[('standardscalerwrapper', StandardScalerWrapper(copy=True, with_mean=False, with_std=False)), ('xgboostclassifier', XGBoostClassifier(booster='gbtree', colsample_bytree=1, eta=0.05, gamma=0, max_depth=6, max_leaves=0, n_estimators=200, n_jobs=1, objective='reg:logistic', problem_info=ProblemInfo(gpu_training_param_dict={'processing_unit_type': 'cpu'}), random_state=0, reg_alpha=0.625, reg_lambda=0.8333333333333334, subsample=0.8, tree_method='auto'))], verbose=False)), ('31', Pipeline(memory=None, steps=[('standardscalerwrapper', StandardScalerWrapper(copy=True, with_mean=F

In [15]:
print(fitted_model)

Pipeline(memory=None,
         steps=[('datatransformer',
                 DataTransformer(enable_dnn=False, enable_feature_sweeping=True, feature_sweeping_config={}, feature_sweeping_timeout=86400, featurization_config=None, force_text_dnn=False, is_cross_validation=True, is_onnx_compatible=False, observer=None, task='classification', working_dir='/mnt/batch/tasks/shared/LS_root/mount...
                 PreFittedSoftVotingClassifier(classification_labels=array([0, 1]), estimators=[('24', Pipeline(memory=None, steps=[('standardscalerwrapper', StandardScalerWrapper(copy=True, with_mean=False, with_std=False)), ('xgboostclassifier', XGBoostClassifier(booster='gbtree', colsample_bytree=1, eta=0.05, gamma=0, max_depth=6, max_leaves=0, n_estimators=200, n_jobs=1, objective='reg:logistic', problem_info=ProblemInfo(gpu_training_param_dict={'processing_unit_type': 'cpu'}), random_state=0, reg_alpha=0.625, reg_lambda=0.8333333333333334, subsample=0.8, tree_method='auto'))], verbose=False)), ('